### Improving Semantic Water Segmentation by Fusing Sentinel-1 Intensity and Interferometric Synthetic Aperture Radar (InSAR) Coherence Data

**Author: Ernesto Colon**
**The Cooper Union for the Advancement of Science and Art**

#### Unet-2D Model Training

Import libraries

In [1]:
import sys
sys.path.append('..')
import tensorflow as tf
import time
from utils import dataset_gen

# check that a GPU is enabled
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

!nvidia-smi

script_start_time = time.time()

Found GPU at: /device:GPU:0
Sun Mar 13 12:35:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.11       Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 32%   47C    P2    46W / 350W |   1777MiB / 24576MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

Load the train, validation, and test dataframes

In [2]:
# Define dictionary with filepaths
base_dir = r"C:\Users\ernes\OneDrive - The Cooper Union for the Advancement of Science and Art\Thesis-DESKTOP-UID4081\flood_mapping\10m_resolution"

train_val_test_pths = {'train_fn_df' : f"{base_dir}\\ds_train_split_10m.csv",
                       'val_fn_df' : f"{base_dir}\\ds_val_split_10m.csv",
                       'test_fn_df' : f"{base_dir}\\ds_test_split_10m.csv"}

train_val_fn_df, test_fn_df, train_size, val_size, test_size =\
    dataset_gen.unet_load_ds_df(train_val_test_pths['train_fn_df'],
                                train_val_test_pths['val_fn_df'],
                                train_val_test_pths['test_fn_df'])

Generate data sets

In [3]:
# Define dictionaries to hold the datasets - the keys will be the different scenarios
X_train_dict = {}
Y_train_dict = {}

X_val_dict = {}
Y_val_dict = {}

X_test_dict = {}
Y_test_dict = {}

Y_pred_dict = {}

# Define scenario number to scenario name mapping
scenario_dict = {1: 'co_event_intensity_only',
                 2: 'pre_co_event_intensity',
                 3: 'pre_co_event_int_coh'}

scenario_num_bands = {1: 2,
                      2: 4,
                      3: 6}

# Define the number of bands per scenario
num_bands_dict = {'co_event_intensity_only': 2,
                 'pre_co_event_intensity': 4,
                 'pre_co_event_int_coh': 6}

IMG_SIZE = 512

# define dictionaries to hold the datasets
train_val_samples_dict = {}
test_samples_dict = {}

scenarios = [1, 2, 3]

for scenario in scenarios:

    # Create the samples list given the dataframes with file paths as input
    train_val_samples_dict[f"scenario_{scenario}"], test_samples_dict[f"scenario_{scenario}"] = \
        dataset_gen.create_samples_list({'scenario': scenario_dict[scenario],
                                            'test_df': test_fn_df,
                                            'train_val_df': train_val_fn_df})

    # Create data sets dictionary
    X_train_dict[f"scenario_{scenario}"], X_val_dict[f"scenario_{scenario}"], X_test_dict[f"scenario_{scenario}"] =\
        dataset_gen.unet_ds_creation({'train_val_list': train_val_samples_dict[f"scenario_{scenario}"],
                                      'test_list': test_samples_dict[f"scenario_{scenario}"]})

    # Batch the tensorflow train, val, and test data set generators
    X_train_dict[f"scenario_{scenario}"] = X_train_dict[f"scenario_{scenario}"].batch(5).prefetch(tf.data.experimental.AUTOTUNE)
    X_val_dict[f"scenario_{scenario}"] = X_val_dict[f"scenario_{scenario}"].batch(5).prefetch(tf.data.experimental.AUTOTUNE)
    X_test_dict[f"scenario_{scenario}"] = X_test_dict[f"scenario_{scenario}"].batch(1)


Trying out the attention U-net from this repo:

https://github.com/yingkaisha/keras-unet-collection

In [4]:
from keras_unet_collection import models
# create dictionary to hold the models by scenario
unet_2d_models = {}

In [5]:
# loop through scenarios and generate the models
for scenario in scenarios:
    # Create models for each scenario
    print("\n*******************************************\n")
    print(f"Generating model for scenario: {scenario}")
    unet_2d_models[f"scenario_{scenario}"] = models.u2net_2d((IMG_SIZE, IMG_SIZE, scenario_num_bands[scenario]),
                                                             n_labels=2,
                                                             filter_num_down=[64, 128, 256, 512, 1024],
                                                             filter_num_up='auto',
                                                             filter_mid_num_down='auto',
                                                             filter_mid_num_up='auto',
                                                             filter_4f_num='auto',
                                                             filter_4f_mid_num='auto',
                                                             activation='ReLU',
                                                             output_activation='Sigmoid',
                                                             batch_norm=False,
                                                             pool=True,
                                                             unpool=True,
                                                             deep_supervision=False,
                                                             name='u2net')
    print("*******************************************")

#unet_2d_models['scenario_3'].summary()


*******************************************

Generating model for scenario: 1
Automated hyper-parameter determination is applied with the following details:
----------
	Number of RSU output channels within downsampling blocks: filter_num_down = [64, 128, 256, 512, 1024]
	Number of RSU intermediate channels within downsampling blocks: filter_mid_num_down = [16, 32, 64, 128, 256]
	Number of RSU output channels within upsampling blocks: filter_num_up = [64, 128, 256, 512, 1024]
	Number of RSU intermediate channels within upsampling blocks: filter_mid_num_up = [16, 32, 64, 128, 256]
	Number of RSU-4F output channels within downsampling and bottom blocks: filter_4f_num = [1024, 1024]
	Number of RSU-4F intermediate channels within downsampling and bottom blocks: filter_4f_num = [512, 512]
----------
Explicitly specifying keywords listed above if their "auto" settings do not satisfy your needs
----------
The depth of u2net_2d = len(filter_num_down) + len(filter_4f_num) = 7
******************

In [6]:
# Define a learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.0001,
                                                             decay_steps=200,
                                                             decay_rate=0.96,
                                                             staircase=True)

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule,
                                     beta_1=0.9,
                                     beta_2=0.999,
                                     epsilon=1e-07,
                                     amsgrad=False,
                                     name='Adam')

unet_2d_train_hist = {}

### Scenario 1 - Co-event Intensity Model

In [7]:
# Compile the model
current_scenario = 1
unet_2d_models[f"scenario_{current_scenario}"].compile(optimizer=optimizer,
                                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                               metrics=['accuracy'])

# Start training routine
train_start_time = time.time()

EPOCHS = 1
unet_2d_train_hist[f"scenario_{current_scenario}"] =\
    unet_2d_models[f"scenario_{current_scenario}"].fit(X_train_dict[f"scenario_{current_scenario}"],
                                               validation_data=X_val_dict[f"scenario_{current_scenario}"],
                                               epochs=EPOCHS)

print("--- %s seconds ---" % (time.time() - train_start_time))

 16/444 [>.............................] - ETA: 3:54 - loss: 0.3758 - accuracy: 0.8833

KeyboardInterrupt: 

Save the model weights

In [ ]:
unet_2d_model_pth = r"D:\SAR_data\Models\refactored\10m_res\unet_2d"
unet_2d_models[f"scenario_{current_scenario}"].save_weights(
    f"{unet_2d_model_pth}\\scenario_{current_scenario}" + "\\" + f"unet2d_10m_{scenario_dict[current_scenario]}")


### Scenario 2 - Pre-event and Co-event Intensity Model

In [ ]:
# Compile the model
current_scenario = 2
unet_2d_models[f"scenario_{current_scenario}"].compile(optimizer=optimizer,
                                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                               metrics=['accuracy'])

# Start training routine
train_start_time = time.time()

EPOCHS = 30
unet_2d_train_hist[f"scenario_{current_scenario}"] =\
    unet_2d_models[f"scenario_{current_scenario}"].fit(X_train_dict[f"scenario_{current_scenario}"],
                                               validation_data=X_val_dict[f"scenario_{current_scenario}"],
                                               epochs=EPOCHS)

print("--- %s seconds ---" % (time.time() - train_start_time))


Save the model weights for scenario 2

In [ ]:
unet_2d_models[f"scenario_{current_scenario}"].save_weights(
    f"{unet_2d_model_pth}\\scenario_{current_scenario}" + "\\" + f"unet2d_10m_{scenario_dict[current_scenario]}")


### Scenario 3 - Pre-event and Co-event Intensity Model

In [7]:
# Compile the model
current_scenario = 3
unet_2d_models[f"scenario_{current_scenario}"].compile(optimizer=optimizer,
                                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                               metrics=['accuracy'])

# Start training routine
train_start_time = time.time()

EPOCHS = 30
unet_2d_train_hist[f"scenario_{current_scenario}"] =\
    unet_2d_models[f"scenario_{current_scenario}"].fit(X_train_dict[f"scenario_{current_scenario}"],
                                               validation_data=X_val_dict[f"scenario_{current_scenario}"],
                                               epochs=EPOCHS)

print("--- %s seconds ---" % (time.time() - train_start_time))


Epoch 1/30


ResourceExhaustedError:  OOM when allocating tensor with shape[10,192,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model_2/u2net_up_4_in_0/Conv2D/Conv2DBackpropInput (defined at \AppData\Local\Temp/ipykernel_21864/4261610726.py:12) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_27876]

Function call stack:
train_function


Save the model weights for scenario 3

In [ ]:
unet_2d_models[f"scenario_{current_scenario}"].save_weights(
    f"{unet_2d_model_pth}\\scenario_{current_scenario}" + "\\" + f"unet2d_10m_{scenario_dict[current_scenario]}")